<a href="https://colab.research.google.com/github/ph20d204/Instrument_Automation/blob/main/LifeTimeMeasurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import inspect,dis,dill
import sys
from time import *
import pulsestreamer
from pulsestreamer import PulseStreamer,findPulseStreamers,OutputState,TriggerStart,Sequence

In [ ]:
IPaddress = findPulseStreamers(search_serial='')[0][0]
pulser = PulseStreamer(IPaddress)

In [27]:
delay=input('Prefered Delay time for Pulse Streamer for one continuous time_length : ')
laser_on=input('Laser on time : '); laser_off=input('Laser off time : ')
read_on=input('Counter on time : ')
guess_repeats=input('Prefered number of guess_repeats of one laser_block : '); averages=input('The number of averages : ')
n_runs=input('No of times the complete Sequence runs : ')
laser_port=input('Laser port No : '); ref_port=input('Reference Counter port No : '); read_port=input('Read Counter port No : ')

prefered Delay time for Pulse Streamer for one continuous time_length: 50
Laser on time: 1000
Laser off time: 3000
counter on time: 150
prefered number of guess_repeats of one laser_block: 20
the number of averages: 10
Laser port No: 0
Reference port No: 1
Counter port No: 2


In [16]:
def lifetime(pulser,nidaq,introduced_delay,laser_on,laser_off,read_on,guess_repeats,averages,n_runs,*ports):  # all in nanosecond range

  ## Checking the initial condition of Pulse Streamer
  if pulser.hasSequence()!=0:
    print('Pulse Streamer has a Sequence. Do you want to proceed?\n')
    permission = int(input('Press 0/1 : \n'))
  if permission==0:
    pass

  ## Producing the Patterns
  init_laser_off = introduced_delay ; end_laser_off = introduced_delay
  laser_block = [(laser_on,1),(laser_off,0)]
  laser_pattern = [(init_laser_off,0)]; ref_pattern = [(init_laser_off,0)]; read_pattern = [(init_laser_off,0)]
  for t in range(int(guess_repeats)):
    position = laser_on+t*read_on; remain = laser_off-t*read_on-read_on
    if remain<=0:
      break
    laser_pattern += laser_block
    ref_pattern += [(laser_on,0),(read_on,1),(laser_off-read_on,0)]
    read_pattern += [(position,0),(read_on,1),(remain,0)]
  laser_pattern +=[(end_laser_off,0)]; ref_pattern +=[(end_laser_off,0)]; read_pattern += [(end_laser_off,0)]

  ## Producing the Sequences
  reSet = pulser.reset()
  if reSet==0:
    print('Pulse Streamer has been reset.\n')
  seq = pulser.createSequence()
  seq.setDigital(ports[0],laser_pattern) ; seq.setDigital(ports[1],ref_pattern) ; seq.setDigital(ports[3],read_pattern)
  print(f'Plot of the Sequence without averaging:\n {seq.plot()}\n')
  seq*=averages

  ## Checking the Conditions
  if pulser.hasSequence()!=0:
    print('Pulse Streamer has no Sequence uploaded!')
    pass
  if seq.isEmpty()==0:
    print('Sequence to Pulse Streamer is empty!')
    pass
  if seq.getDuration()%8==0:
    print("Sequence duration is not multiple of 8ns!\n Please check by 'seq.get_pad()' method.")
    pass

  ## Performing the Streaming
  pulser.stream(seq,n_runs=n_runs,final=([],0,0))
  if pulser.hasFinished()==1:
    print('Streaming has been finished.\n')
  reSet = pulser.reset()
  if reSet==0:
    print('Pulse Streamer has been reset after Streaming.\n')
  # return laser_pattern,ref_pattern,read_pattern

In [66]:
# ## Choosing the Devices
# use_pulser=input('Enter your specific Pulse Streamer: ')
# use_nidaq=input('Enter your specific NIDAQmx: ')

lifetime(pulser,_,delay,laser_on,laser_off,read_on,guess_repeats,averages,n_runs,laser_port,ref_port,read_port)

1